# Pro-tips- the challenge here is that with the netcdf in its origional configuration it was not possible to read all the data in to downsample it. So what we did was to read in each file, swap the time and obs dimensions and then only keep the 
1. Create a directory in you data files for downsampled .ncdfs to live

In [1]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import re
import requests
import pickle as pk
import os
import gc
#!pip install netcdf4 == 1.5.0

In [2]:
# make the output directory
new_dir = '/home/jovyan/data/botpt/minute_mean_dataD/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [3]:
# Sensor E: url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T104012-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# Sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181020T213838-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
# Sensor C url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T041943-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/catalog.html'

In [ ]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i + '#fillmismatch') for i in x]
#datasets

# pd routine

In [ ]:
%%time
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = '/home/jovyan/data/botpt/minute_mean_dataD/' + i.split('/')[-1][:-3] + '_resampled' + '.nc'
    ds = xr.Dataset.from_dataframe(pressure_min)
    ds.to_netcdf(out)

# Build Dask Cluster
1. Use gui interface to create a new cluster with ~10 workers
2. Use < > to insert an "import Client" statement. This is critical because it is how your script knows to use yoru cluster. 
3. Execute client cell
4. Execute your Dask Cell. Note that once you have a cluster running you do not need to re-import the client. 
5. When finished, always shut down your cluster. 

In [2]:
from dask.distributed import Client

client = Client("tcp://10.0.129.130:36559")
client

Client Scheduler: tcp://10.0.129.130:36559 Dashboard: /user/daxsoule/proxy/8787/status,Cluster Workers: 14 Cores: 14 Memory: 17.50 GB


In [4]:
ds = xr.open_mfdataset('/home/jovyan/data/botpt/minute_mean_dataD/*_resampled.nc', parallel=True
                       ,combine='by_coords')
ds

<xarray.Dataset>
Dimensions:          (index: 1145599)
Coordinates:
  * index            (index) datetime64[ns] 2017-06-13 ... 2019-10-14T23:59:00
Data variables:
    bottom_pressure  (index) float32 dask.array<chunksize=(10800,), meta=np.ndarray>

In [5]:
df = ds.to_dask_dataframe()

In [6]:
df.head()

,index,bottom_pressure
0,2017-06-13 00:00:00,2259.683105
1,2017-06-13 00:01:00,2259.678711
2,2017-06-13 00:02:00,2259.674072
3,2017-06-13 00:03:00,2259.660400
4,2017-06-13 00:04:00,2259.666992


In [7]:
df.tail()

,index,bottom_pressure
1145594,2019-10-14 23:55:00,2258.016357
1145595,2019-10-14 23:56:00,2258.004395
1145596,2019-10-14 23:57:00,2257.993408
1145597,2019-10-14 23:58:00,2257.980957
1145598,2019-10-14 23:59:00,2257.965576
